In [1]:
%%bash
chia keys derive -f 4001300365 wallet-address -n 5

Wallet address 0: txch1zgwteqg3gaqm88mu764xrqps8gh4lm5ape008xr2qg2u5345zwzswr3wvy
Wallet address 1: txch1va46l8pd2c2h2djs5jved3hfqkh0384tcda8347xt4u6rfea02lsztjjmv
Wallet address 2: txch15c59kptqzxj9s0jqwm9035pkldg3qcy42kl7czsu6wyp465twwlsjcdehn
Wallet address 3: txch1tngsu0rj98c2c077rdkdhxt9pljtzg9mq2rxy3jwl769ltxe43lqskcu9h
Wallet address 4: txch1dagw2tqqfzzfy0krhqvp9dlgrmr7e774cjlfjlutyqtkas67j44sxu3afy


In [2]:
%%bash
chia wallet show -f 4001300365

Wallet height: 8
Sync status: Synced
Balances, fingerprint: 4001300365

Chia Wallet:
   -Total Balance:         1.999499999997 txch (1999499999997 mojo)
   -Pending Total Balance: 1.999499999997 txch (1999499999997 mojo)
   -Spendable:             1.999499999997 txch (1999499999997 mojo)
   -Type:                  STANDARD_WALLET
   -Wallet ID:             1

Profile 1:
   -Total Balance:         1.0  
   -Pending Total Balance: 1.0  
   -Spendable:             1.0  
   -Type:                  DECENTRALIZED_ID
   -DID ID:                did:chia:1fcs44905vc749sdcnqem4xh3fzmmx65p7htrz4p2rnrhe9m4jvgqy5j9pa
   -Wallet ID:             2

Profile 2:
   -Total Balance:         1.0  
   -Pending Total Balance: 1.0  
   -Spendable:             1.0  
   -Type:                  DECENTRALIZED_ID
   -DID ID:                did:chia:1u7mstlcyenx22jnzyucnhzrnzecg0gr7996n0alkncl09gdnd0pq0fz0c8
   -Wallet ID:             4

NFT Wallet:
   -Total Balance:         0.0  
   -Pending Total Balance: 0.0  


In [3]:
import aiosqlite
from pathlib import Path
DB_VERSION = 2
DB_PATH = Path(f'/Users/karlkim/.chia/simulator/hints/db/blockchain_v2_simulator0.sqlite')

In [4]:
from chia.consensus.block_record import BlockRecord
query = f"""SELECT coin_id, hint from hints"""
async with aiosqlite.connect(DB_PATH) as db:
    async with db.execute(query) as cursor:
        async for row in cursor:
            coin_id = row[0]
            hint = row[1]
            addr = encode_puzzle_hash(hint, 'txch')
            print(f'coin_id: {coin_id.hex()}')
            print(f'hint:\t {hint.hex()}')
            print(f'addr:\t {addr}\n')

coin_id: bee957687d5e5ef69537a99f125146e267c722c0e2a0d427e7e3d788c2443c0c
hint:	 676baf9c2d5615753650a49996c6e905aef89eabc37a78d7c65d79a1a73d7abf
addr:	 txch1va46l8pd2c2h2djs5jved3hfqkh0384tcda8347xt4u6rfea02lsztjjmv

coin_id: 552add06fcb8986371bc3892233a3d29021377220411a072766486f005d7b42c
hint:	 5cd10e3c7229f0ac3fde1b6cdb99650fe4b120bb028662464effb45facd9ac7e
addr:	 txch1tngsu0rj98c2c077rdkdhxt9pljtzg9mq2rxy3jwl769ltxe43lqskcu9h

coin_id: 83d36e93ca4255b388143d4193a0ce1d8399cdc0fa2ee277f50b8bf4dbc96a47
hint:	 121cbc81114741b39f7cf6aa6180303a2f5fee9d0e5ef3986a0215ca46b41385
addr:	 txch1zgwteqg3gaqm88mu764xrqps8gh4lm5ape008xr2qg2u5345zwzswr3wvy

coin_id: 4c57566e626efb6f857b80e3d29cc3082b8712340480ddfd7370379cfb3a6a95
hint:	 5cd10e3c7229f0ac3fde1b6cdb99650fe4b120bb028662464effb45facd9ac7e
addr:	 txch1tngsu0rj98c2c077rdkdhxt9pljtzg9mq2rxy3jwl769ltxe43lqskcu9h

